# MongoDB Inventory

## Container starten

Docker container starten mit dem Kernel PowerShell, danach auf den Kernel Python 3 switchen

In [1]:
docker start mongodb1
docker ps

mongodb1


CONTAINER ID   IMAGE     COMMAND                  CREATED      STATUS              PORTS                                           NAMES
354fa0302bfe   mongo     "docker-entrypoint.s…"   4 days ago   Up About a minute   0.0.0.0:27017->27017/tcp, :::27017->27017/tcp   mongodb1


## Connection String

Connection String mit localhost und default Port abfüllen

In [18]:
# <-- evaluate localhost --> #
import pymongo, dns, os, sys, csv, json, uuid
from datetime import datetime

if sys.platform == "linux" or sys.platform == "linux2":
    home_dir = os.environ['HOME']
    connectionstring = "mongodb://0.0.0.0:27017"
elif sys.platform == "darwin":
    home_dir = os.environ['HOME']
    connectionstring = "mongodb://0.0.0.0:27017"
elif sys.platform == "win32":
    home_dir  = os.environ['USERPROFILE']
    connectionstring = "mongodb://localhost:27017"

# Join the path to Downloads
downloads = os.path.join(home_dir,'Downloads')

print(connectionstring)

mongodb://localhost:27017


## Database

Connect to MongoDB and list all database

In [2]:
# <-- run this function on a mac --> #
def get_dbs(connectionstring):
    '''Connect to MongoDB and list all database'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    print(mongo_client.list_database_names())
    mongo_client.close()

get_dbs(connectionstring)

['admin', 'config', 'local', 'tinu']


## Documents

### Remove documents

Connect to MongoDB and delete all documents query-based

In [27]:
document_toremove = {'DeviceName':'ThinkPad'}
def delete_all_documents(connectionstring, database, collection, query, output = False):
    """Connect to MongoDB and delete all documents

    Args:
        connectionstring (String): Connection-string of the MongoDB
        database (String): Database name to export the data
        collection (String): Collection name to export the data
        query (JSON-String): Search for 
        output (bool, optional): Print output. Defaults to False.

    Example:
        credentials = input('user:password')
        connectionstring = "mongodb+srv://"+credentials+"@cluster0.epl3x.mongodb.net/?retryWrites=true&w=majority"
        delete_collection(connectionstring, 'JupyterNB', 'PoweredOffVMs', {'address':'India'}, True)
    """
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[database]
    mongo_col    = mongo_db[collection]
    mongo_col.delete_many(query)
    
    if output:
        for doc in mongo_col.find():
            print(f'{doc["ID"]}\t{doc["RunTime"]}\t{doc["DeviceType"]}\t{doc["DeviceName"]}')

    mongo_client.close()

delete_all_documents(connectionstring, 'tinu', 'inventory', document_toremove, True)

96fb423e-f36b-1410-8134-00daf9a34b4e	2021-05-21T15:00:24.09	Convertible	Microsoft Surface Pro 4
97fb423e-f36b-1410-8134-00daf9a34b4e	2021-05-22T17:04:13.177	iMac	Apple iMac (Retina 4K, 21.5-inch, 2017)
4a27433e-f36b-1410-8136-00daf9a34b4e	2021-05-22T17:38:48.047	Handy	Huawei P30 Pro DualSIM 4G+ Aurora
4e27433e-f36b-1410-8136-00daf9a34b4e	2021-05-21T16:30:57.48	Camera	NIKON Z6
82c7423e-f36b-1410-8138-00daf9a34b4e	2021-05-22T17:23:32.503	Convertible	HP EliteBook x360 1030 G8


### Import JSON

In [38]:
file_toimport = 'MssqlDB.tinu-inventory-2021523-13-00-00.json'

def import_json(json_file, connectionstring, database, collection):
    """Import data from JSON-file

    Args:
        json_file (String): FullName of the JSON-file to import
        connectionstring (String): Connection-string of the MongoDB
        database (String): Database name to import the data
        collection (String): Collection name to import the data
        
    Example:
        credentials = input('user:password')
        connectionstring = "mongodb+srv://"+credentials+"@cluster0.epl3x.mongodb.net/?retryWrites=true&w=majority"
        imp_json_file = "C:\\Users\\Tinu\\Downloads\\poweredOffVMs.json"
        import_json(imp_json_file, connectionstring, 'JupyterNB', 'PoweredOffVMs')
    """
    check_path = os.path.exists(json_file)
    if(check_path == True):
        import pymongo
        mongo_client = pymongo.MongoClient(connectionstring)
        mongo_db     = mongo_client[database]
        mongo_col    = mongo_db[collection]

        with open(json_file, "r") as read_file:
            json_data = json.load(read_file)

        # It is important to remove the _id field in order to import it into mongodb
        for element in json_data:
            element.pop('_id', None)

        try:
            if isinstance(json_data, list):
                mongo_col.insert_many(json_data)  
            else:
                mongo_col.insert_one(json_data)
            
        except:
            sys.exit('file {}'.format(json_file))
                
        mongo_client.close()

    else:
        print(f'[WARN] File not found: {json_file}')

json_file = os.path.join(downloads, file_toimport)
print(json_file)
import_json(json_file, connectionstring, 'tinu', 'inventory')

C:\Users\info\Downloads\MssqlDB.tinu-inventory-2021523-13-00-00.json


Query for document

In [39]:
filter_set = {'ID' : '2f80f971-e795-4127-b606-8161d4e2176e'} 

def query_document(connectionstring, mongodatabase, collection, filterset):
    '''Connect to MongoDB and filter a document'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[mongodatabase]
    mongo_col    = mongo_db[collection]
    mongo_client.close()
    return mongo_col.find(filterset)

import pandas as pd
table = []

result = query_document(connectionstring, 'tinu', 'inventory', filter_set)
for doc in result:
    #print(doc)
    thisdict = { 
        'ID'             : doc['ID'],
        'RunTime'        : doc['RunTime'],
        'DeviceType'     : doc['DeviceType'],
        'DeviceName'     : doc['DeviceName'],
        'Physical cores' : doc['Physical cores'],
        'MemoryGB'       : doc['MemoryGB'],
        'DiskspaceGB'    : doc['DiskspaceGB'],
        'PurchaseDate'   : doc['PurchaseDate'],
        'Price'          : doc['Price'],
        'Payment'        : doc['Payment'],
        'Warranty'       : doc['Warranty'],
        'Link'           : doc['Link'],
    }
    table.append(thisdict)

df = pd.DataFrame(table)
df.head()

,ID,RunTime,DeviceType,DeviceName,Physical cores,MemoryGB,DiskspaceGB,PurchaseDate,Price,Payment,Warranty,Link
0,2f80f971-e795-4127-b606-8161d4e2176e,2021-06-05T09:13:33,Notebook,ThinkPad,4,16,512,2021-06-05T09:13:33,1199.00,Mastercard,3+1,https://tinuwalther.github.io


### List all documents

In [36]:
# <-- Print max 1000 documents --> #
def get_documents(connectionstring, database, collection):
    '''Connect to MongoDB and return data'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[database]
    mongo_col    = mongo_db[collection]
    mongo_client.close()
    return mongo_col.find()

import pandas as pd
table = []

result = get_documents(connectionstring, 'tinu', 'inventory')
for doc in result:
    #print(doc)
    thisdict = { 
        'ID'             : doc['ID'],
        'RunTime'        : doc['RunTime'],
        'DeviceType'     : doc['DeviceType'],
        'DeviceName'     : doc['DeviceName'],
        'Physical cores' : doc['Physical cores'],
        'MemoryGB'       : doc['MemoryGB'],
        'DiskspaceGB'    : doc['DiskspaceGB'],
        'PurchaseDate'   : doc['PurchaseDate'],
        'Price'          : doc['Price'],
        'Payment'        : doc['Payment'],
        'Warranty'       : doc['Warranty'],
        'Link'           : doc['Link'],
    }
    table.append(thisdict)

df = pd.DataFrame(table)
df.head(1000)

,ID,RunTime,DeviceType,DeviceName,Physical cores,MemoryGB,DiskspaceGB,PurchaseDate,Price,Payment,Warranty,Link
0,96fb423e-f36b-1410-8134-00daf9a34b4e,2021-05-21T15:00:24.09,Convertible,Microsoft Surface Pro 4,2.0,8.0,256.0,2015-05-13T00:00:00,1099.00,Maestrocard,2,None
1,97fb423e-f36b-1410-8134-00daf9a34b4e,2021-05-22T17:04:13.177,iMac,"Apple iMac (Retina 4K, 21.5-inch, 2017)",4.0,16.0,512.0,2019-07-01T00:00:00,2341.4,Mastercard,2+1,None
2,4a27433e-f36b-1410-8136-00daf9a34b4e,2021-05-22T17:38:48.047,Handy,Huawei P30 Pro DualSIM 4G+ Aurora,8.0,8.0,128.0,2019-05-13T00:00:00,999.0,Mastercard,2+1,None
3,4e27433e-f36b-1410-8136-00daf9a34b4e,2021-05-21T16:30:57.48,Camera,NIKON Z6,NaN,NaN,NaN,2020-01-15T00:00:00,2099.0,Mastercard,3+1,None
4,82c7423e-f36b-1410-8138-00daf9a34b4e,2021-05-22T17:23:32.503,Convertible,HP EliteBook x360 1030 G8,4.0,16.0,512.0,2021-05-28T07:45:00,2249.0,Mastercard,3+1,https://store.hp.com/SwitzerlandStore/Merch/Pr...
5,2f80f971-e795-4127-b606-8161d4e2176e,2021-06-05T09:13:33,Notebook,ThinkPad,4.0,16.0,512.0,2021-06-05T09:13:33,1199.00,Mastercard,3+1,https://tinuwalther.github.io


### Update document

In [34]:
filter_set = {'ID' : '2f80f971-e795-4127-b606-8161d4e2176e'} 
update_set = {'$set' : {'Price' : '1199.00'}}

def update_document(connectionstring, mongodatabase, collection, filterset, updateset):
    '''Connect to MongoDB and update a document'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[mongodatabase]
    mongo_col    = mongo_db[collection]

    doc = mongo_col.find_one_and_update(filterset, updateset, upsert=True)
    print(doc)

    mongo_client.close()

update_document(connectionstring, 'tinu', 'inventory', filter_set, update_set)

{'_id': ObjectId('60bb241d98e614c98f675a10'), 'ID': '2f80f971-e795-4127-b606-8161d4e2176e', 'RunTime': '2021-06-05T09:13:33', 'DeviceType': 'Notebook', 'DeviceName': 'ThinkPad', 'Physical cores': 4, 'MemoryGB': 16, 'DiskspaceGB': 512, 'PurchaseDate': '2021-06-05T09:13:33', 'Price': 1234, 'Payment': 'Mastercard', 'Warranty': '3+1', 'Link': 'https://tinuwalther.github.io'}


### Insert one document

In [29]:
now  = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
guid = str(uuid.uuid4())

documenttoimport = { 
    "ID"            : guid,
    "RunTime"       : now, 
    "DeviceType"    : "Notebook",
    "DeviceName"    : "ThinkPad",
    "Physical cores": 4,
    "MemoryGB"      : 16,
    "DiskspaceGB"   : 512,
    "PurchaseDate"  : now,
    "Price"         : 1234,
    "Payment"       : "Mastercard",
    "Warranty"      : "3+1",
    "Link"          : "https://tinuwalther.github.io"
}

def add_document(connectionstring, mongodatabase, collection, document):
    '''Connect to MongoDB and add a document'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[mongodatabase]
    mongo_col    = mongo_db[collection]
    mongo_col.insert_one(document)
    for doc in mongo_col.find():
        print(f'{doc["ID"]}\t{doc["RunTime"]}\t{doc["DeviceType"]}\t{doc["DeviceName"]}')

    mongo_client.close()

add_document(connectionstring, 'tinu', 'inventory', documenttoimport)

96fb423e-f36b-1410-8134-00daf9a34b4e	2021-05-21T15:00:24.09	Convertible	Microsoft Surface Pro 4
97fb423e-f36b-1410-8134-00daf9a34b4e	2021-05-22T17:04:13.177	iMac	Apple iMac (Retina 4K, 21.5-inch, 2017)
4a27433e-f36b-1410-8136-00daf9a34b4e	2021-05-22T17:38:48.047	Handy	Huawei P30 Pro DualSIM 4G+ Aurora
4e27433e-f36b-1410-8136-00daf9a34b4e	2021-05-21T16:30:57.48	Camera	NIKON Z6
82c7423e-f36b-1410-8138-00daf9a34b4e	2021-05-22T17:23:32.503	Convertible	HP EliteBook x360 1030 G8
2f80f971-e795-4127-b606-8161d4e2176e	2021-06-05T09:13:33	Notebook	ThinkPad


## PowerShell

In [5]:
if(-not(Get-Module -Name Mdbc)){
    Import-Module -Name Mdbc
}
$DatabaseName   = 'tinu'
$CollectionName = 'inventory'
$mongo_client   = Connect-Mdbc mongodb://localhost
$mongo_db       = Get-MdbcDatabase -Name $DatabaseName
$mongo_col      = Get-MdbcCollection -Database $mongo_db -Name $CollectionName

In [33]:
Get-MdbcDatabase -Client $mongo_client | Select-Object -ExpandProperty DatabaseNamespace


DatabaseName
------------
admin       
config      
local       
tinu        




In [38]:
Get-MdbcData -Collection $mongo_col -As PS | Format-Table -Property RunTime,DeviceType,DeviceName,PurchaseDate,Price,Payment


RunTime                 DeviceType  DeviceName                              PurchaseDate        Price   Payment    
-------                 ----------  ----------                              ------------        -----   -------    
2021-05-21T15:00:24.09  Convertible Microsoft Surface Pro 4                 2015-05-13T00:00:00 1099.00 Maestrocard
2021-05-22T17:04:13.177 iMac        Apple iMac (Retina 4K, 21.5-inch, 2017) 2019-07-01T00:00:00 2341.4  Mastercard 
2021-05-22T17:38:48.047 Handy       Huawei P30 Pro DualSIM 4G+ Aurora       2019-05-13T00:00:00 999     Mastercard 
2021-05-21T16:30:57.48  Camera      NIKON Z6                                2020-01-15T00:00:00 2099    Mastercard 
2021-05-22T17:23:32.503 Convertible HP EliteBook x360 1030 G8               2021-05-28T07:45:00 2249    Mastercard 




In [37]:
$filter_json = @{'ID' = '96fb423e-f36b-1410-8134-00daf9a34b4e'} 
$update_json = @{'$set' = @{'Price' = '1099.00'}}
Update-MdbcData -Collection $mongo_col -Filter $filter_json -Update $update_json

## Container stoppen

Kernel PowerShell switchen und docker container stoppen

In [39]:
docker stop mongodb1

mongodb1
